In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
#Load CSV to DF
weather_df = pd.read_csv('./cities.csv')

In [4]:
#Humidity HM
fig = gmaps.figure(layout={'width': '600x', 'height': '500px'})

locations = weather_df[['Lat','Lng']]
humidity = weather_df['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='500px', width='600x'))

In [5]:
ideal_weather_df = weather_df.loc[
    (weather_df['Max Temp']<=85) & 
    (weather_df['Max Temp']>=70) & 
    (weather_df['Wind Speed']<=6) & 
    (weather_df['Humidity']<=65) &
    (weather_df['Cloudiness']>=25)
]
     
print(len(ideal_weather_df))
ideal_weather_df

7


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88
322,322,tshikapa,99,CD,1558378849,58,-6.42,20.79,79.62,2.15
443,443,lucapa,65,AO,1558378880,48,-8.42,20.74,78.36,2.73
468,468,san cristobal,75,EC,1558378885,37,-0.39,-78.55,71.60,4.70
469,469,san ramon,75,NI,1558378828,65,12.92,-85.84,77.00,5.55
491,491,bouca,68,CF,1558378889,51,6.51,18.28,83.94,4.12
492,492,bolobo,95,CD,1558378889,57,-2.17,16.23,83.94,2.33


In [8]:
hotels = []

for ea in (np.arange(0,len(ideal_weather_df))):
    city = ideal_weather_df.iloc[ea,0]
    country = ideal_weather_df.iloc[ea,7]
    location = f"{ideal_weather_df.iloc[ea,1]}, {ideal_weather_df.iloc[ea,2]}"
    target_type = 'Hotel'
    target_keyword = 'Hotel'
    target_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&keyword={target_keyword}&type={target_type}&radius=5000&key={g_key}"
    try:
        response = requests.get(target_url).json()
        hotel_name = response['results'][0]['name']
        lat = response['results'][0]['geometry']['location']['lat']
        lng = response['results'][0]['geometry']['location']['lng']    
        hotels.append({'city': city, 'country': country, 'hotel_name': hotel_name, 'hotel_lat': lat, 'hotel_lng': lng})
        
    except:
        print(f"No hotels listed within a 5000m radius of {city} {country}")
        
hotels_df = pd.DataFrame(hotels)
coords = hotels_df[['hotel_lat','hotel_lng']]
names = []
for ea in (np.arange(0,len(hotels_df))):
    names.append("Hotel: "+hotels_df.iloc[ea,2]+", City: " +hotels_df.iloc[ea,0]+", Country: "+hotels_df.iloc[ea,1])
    
markers = gmaps.marker_layer(coords,info_box_content=names, display_info_box=True)
fig.add_layer(markers)

fig

No hotels listed within a 5000m radius of 52 60.78
No hotels listed within a 5000m radius of 322 20.79
No hotels listed within a 5000m radius of 443 20.74
No hotels listed within a 5000m radius of 468 -78.55
No hotels listed within a 5000m radius of 469 -85.84
No hotels listed within a 5000m radius of 491 18.28
No hotels listed within a 5000m radius of 492 16.23


KeyError: "None of [Index(['hotel_lat', 'hotel_lng'], dtype='object')] are in the [columns]"